dataset: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

In [226]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from scipy.special import expit

In [227]:
data = pd.read_csv('diabetes.csv')

In [228]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [229]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [230]:
data['Outcome'].value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [231]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [232]:
X = data.drop('Outcome', axis=1)
y = data['Outcome']

In [233]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [234]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [235]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

# NN

In [237]:
class NeuralNetwork:
    def __init__(self,input_node,hidden_node,output_node,learning_rate):

        self.input_node = input_node
        self.hidden_node = hidden_node
        self.output_node = output_node
        self.learning_rate = learning_rate
        self.activation_function = lambda x:expit(x)
        
        self.w_i_h = np.random.normal(0,pow(self.input_node,-0.5),(self.hidden_node,self.input_node))
        self.w_h_o = np.random.normal(0,pow(self.hidden_node,-0.5),(self.output_node,self.hidden_node))

    def query(self,inputs):

        inputs = np.array(inputs,ndmin=2).T
        x_hidden=np.dot(self.w_i_h,inputs)
        hidden_layer=self.activation_function(x_hidden)
        x_outputs = np.dot(self.w_h_o,hidden_layer)
        output_layer=self.activation_function(x_outputs)

        return output_layer,hidden_layer,inputs

    def train(self,inputs,targets):

        targets=np.array(targets,ndmin=2).T 
        output, hidden , inp =self.query(inputs)
        err_output=targets-output
        err_hidden=np.dot(self.w_h_o.T,err_output)

        self.w_h_o += self.learning_rate * (np.dot(err_output*output*(1-output),hidden.T))
        self.w_i_h += self.learning_rate * (np.dot(err_hidden*hidden*(1-hidden),inp.T))

In [238]:
input_nodes = X_train.shape[1]
hidden_nodes = 100
output_nodes = 1
learning_rate = 0.001

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [239]:
epochs = 500
for e in range(epochs):
    for i in range(X_train.shape[0]):
        nn.train(X_train[i], y_train[i])
    if (e+1) % 50 == 0:
        print(f'Epoch {e+1}/{epochs} done')

Epoch 50/500 done
Epoch 100/500 done
Epoch 150/500 done
Epoch 200/500 done
Epoch 250/500 done
Epoch 300/500 done
Epoch 350/500 done
Epoch 400/500 done
Epoch 450/500 done
Epoch 500/500 done


In [241]:
predictions = []


for i in range(X_test.shape[0]):
    output, _, _ = nn.query(X_test[i])
    predictions.append(1 if output[0,0] > 0.5 else 0)

predictions = np.array(predictions)
accuracy = np.mean(predictions == y_test)


print(f'Test Accuracy: {accuracy:.4f}')

Test Accuracy: 0.6927


# MLP

In [243]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.25),
    Dense(32, activation='relu'),
    Dropout(0.25),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [244]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [245]:
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.4957 - loss: 0.7035 - val_accuracy: 0.6810 - val_loss: 0.6686
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6848 - loss: 0.6490 - val_accuracy: 0.6724 - val_loss: 0.6424
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7109 - loss: 0.6046 - val_accuracy: 0.6897 - val_loss: 0.6055
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6978 - loss: 0.5572 - val_accuracy: 0.6983 - val_loss: 0.5759
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7217 - loss: 0.5204 - val_accuracy: 0.7241 - val_loss: 0.5551
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7587 - loss: 0.4889 - val_accuracy: 0.7241 - val_loss: 0.5369
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7870 - loss: 0.4820 - val_accuracy: 0.7155 - val_loss: 0.5240
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7848 - loss: 0.4663 - val_accuracy: 0.7500 

In [246]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)

print(f'Test Accuracy: {accuracy:.4f}')

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7240 - loss: 0.6960 
Test Accuracy: 0.7240


In [247]:
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")

print(classification_report(y_test, y_pred))

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  
              precision    recall  f1-score   support

           0       0.79      0.78      0.78       123
           1       0.61      0.62      0.62        69

    accuracy                           0.72       192
   macro avg       0.70      0.70      0.70       192
weighted avg       0.72      0.72      0.72       192

